In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

In [ ]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/6z108v0i01cba1sb6rko8jtqz9n87a3t.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data


# get the synthetic images and labels and unzip those into the same directory
!wget -nc -O data/synthetic_images.zip https://duke.box.com/shared/static/bzvtcpzxr7xpfrfuen5w8i35wqb494z0.zip
!wget -nc -O data/synthetic_labels.zip https://duke.box.com/shared/static/svq4b2xpl46qb7xjq6ylq6sozn5v2geu.zip

!unzip -n -q data/synthetic_images.zip -d data
!unzip -n -q data/synthetic_labels.zip -d data


In [ ]:
# delete the original data folder so we can unzip our own data folder with the wind turbine data
!rm -rf yolov3/data

# load in the data.zip file from the box
!wget -nc -O yolov3/data_multiclass.zip https://duke.box.com/shared/static/0hbo1y2lmzx1sr389yvf0viu1wd42a10.zip

!unzip -n -q yolov3/data_multiclass.zip -d yolov3

# Get correct cfg file (multiclass one)
!rm -rf yolov3/cfg
!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/sffuk5yvl6ewla894m001ceyj9qlebjt.zip
!unzip -n -q yolov3/cfg.zip -d yolov3


# get the multiclass syn + real data files from the box
!wget -nc -O yolov3/data_multiclass_adding_synthetic.zip https://duke.box.com/shared/static/umq3nbidnfx8fyv7uui5i7swc5yd0vcq.zip

!unzip -n -q yolov3/data_multiclass_adding_synthetic.zip -d yolov3


In [ ]:
%pwd

In [ ]:
%cd yolov3

In [ ]:
# BEFORE RUNNING train.py: edit the obj hyperparameter on line 29 of train.py to be 120

# Train the model with real overhead images of wind turbines (with two classes: small and large)
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data_multiclass/wnd_px10_seed17.data --img-size 608 --epochs 300 --batch-size 10

In [ ]:
# Test the model using validation set (this set is the same for both normal and for adding synthetic). Compute predictions with the very last set of weights from training
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data_multiclass/wnd_px10_seed17.data --img-size 608 --weights weights/last.pt

In [ ]:
# BEFORE RUNNING train.py: edit the obj hyperparameter on line 29 of train.py to be 120

# Train the model with real overhead images of wind turbines (with two classes: small and large)
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data_multiclass_adding_synthetic/wnd_syn_real_px10_seed17.data --img-size 608 --epochs 300 --batch-size 10

In [ ]:
# Test the model using validation set (this set is the same for both normal and for adding synthetic). Compute predictions with the very last set of weights from training
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data_multiclass_adding_synthetic/wnd_syn_real_px10_seed17.data --img-size 608 --weights weights/last.pt

In [ ]:
!python3 detect.py --names data_multiclass/wnd.names --weights weights/last.pt --source ../data/images --img-size 608